[View in Colaboratory](https://colab.research.google.com/github/TattooeDeer/API_REST_TingoID/blob/master/P2.ipynb)

# Pregunta 2


In [51]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

#!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import numpy as np
np.random.seed(11235813)
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

# Tensorflow & Keras imports
import tensorflow as tf
from keras.layers import Input, RepeatVector, TimeDistributed, Dense, Embedding, Flatten, Activation, Permute, Lambda
from keras.models import Model
from keras import backend as K
from keras.preprocessing import sequence

from google.colab import files
#!git clone https://github.com/TattooeDeer/T3-ANN.git
%cd T3-ANN
!ls


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[Errno 2] No such file or directory: 'T3-ANN'
/content/T3-ANN
Enunciado_T3.ipynb  LICENSE  README.md	Tarea3.ipynb  test_Q.csv  train_Q-A.csv


## a) Carga de los datos en el entorno y análisis descriptivo

In [16]:
train = pd.read_csv('train_Q-A.csv')
test = pd.read_csv('test_Q.csv')

print('Train shape: {0}'.format(train.shape))
print('Test shape: {0}'.format(test.shape))

Train shape: (86821, 3)
Test shape: (11873, 2)


In [17]:
train.head()

,id,question,answer
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s


In [18]:
train.describe()

,id,question,answer
count,86821,86821,86821
unique,86821,86769,64763
top,572fa66704bcaa1900d76b3a,What year did Prince Albert die?,three
freq,1,2,231


Al parecer el primer problema que observamos es que para preguntas cuya respuesta es cuantitativa, la respuesta dada es expresada en algunos casos en palabras y en otros en números, nuestro primer intento de solucionar esto será explorar si se cumplen patrones para cada tipo de respuesta, por ejemplo, solo las fechas se responden con números u otra regla similar.


In [19]:
test.describe()

,id,question
count,11873,11873
unique,11873,11864
top,5ad5316b5b96ef001a10ab76,Who designed Salamanca?
freq,1,2


Se observa algo interesante analizando la cantidad de valores únicos y de conteo de las columnas `question` y  `answer`: La cantidad de ocurrencias únicas no es igual a la cantidad de registros totales, lo que puede indicar que quizás hay preguntas/respuestas repetidas en el dataset. Se observa un comportamiento similar en el conjunto de test aunque en mucho menor medida

In [20]:
print('10 preguntas más populares:\n')
print(train["question"].value_counts().head(10))
print('\n ----------------------------------------------------------\n')
print('10 respuestas más populares:\n')
print(train["answer"].value_counts().head(10))
print('\n ----------------------------------------------------------\n')
print('10 preguntas más populares (Conjunto de test):\n')
print(test["question"].value_counts().head(10))






10 preguntas más populares:

What year did Prince Albert die?                                2
When did Beyonce have her first child?                          2
Who sponsors the Premier League?                                2
What does DRM stand for?                                        2
What does CD stand for?                                         2
Who was Alexander Scriabin's teacher?                           2
How many children did Queen Victoria and Prince Albert have?    2
What was the name of Liszt's mistress?                          2
What year did Chopin leave Warsaw?                              2
What years did the civil war take place?                        2
Name: question, dtype: int64

 ----------------------------------------------------------

10 respuestas más populares:

three    231
two      206
four     171
five     133
six       90
2007      87
2006      85
2010      75
2009      71
seven     71
Name: answer, dtype: int64

 -------------------------------

Las preguntas que más se repiten lo hacen a lo más 2 veces cada una. En cuanto a las respuestas, predominan aquellas que son números escritos como palabras, asi como también números escritos como tal los cuales aprecen ser fechas debido a la cantidad de dígitos y la magnitud que presentan.

Lo anterior puede ser en respuesta a un sesgo en la selección de preguntas.

Finalmente, se debe notar que el conjunto de test está compuesto solo por preguntas, lo que significa que, de utilizar los conjuntos de la forma en la que están, tendremos un entrenamiento supervisado pero tendremos que encontrar una métrica nueva para evaluar el desempeño final de la máquina.

## b) Preprocesamiento

Ahora se procederá a preprocesar ambos conjuntos con el objetivo de mejorar el desempeño que tenga la futura máquina al ingerirlos en el entrenamiento.
Se _tokenizarán_ las preguntas y respuestas del conjunto de entrenamiento y de test, no realizando mayor modificación de las palabras dado que después necesitaremos reconstruir las oraciones.

In [0]:
train_questions = [word_tokenize(sentence.lower()) for sentence in train["question"]] #or processing
test_questions = [word_tokenize(sentence.lower()) for sentence in  test["question"]]
train_answers = [word_tokenize(sentence) for sentence in train["answer"]]


## c) Vocabulario

Ahora, se procede a crear un vocabulario para codificar las palabras en las respuestas a generar, esta aproximación nos servirá para paliar el problema mencionado en el punto *a)*, de que no tenemos las respuestas correctas para el conjunto de test.

In [35]:
# Respuestas
vocab_answer = set()
for sentence in train_answers:
  for word in sentence:
    vocab_answer.add(word)
vocab_answer = ["#end"] + list(vocab_answer)
print('Posibles palabras para respuestas: ', len(vocab_answer))
vocabA_indices = {c: i for i, c in enumerate(vocab_answer)}
indices_vocabA = {c: i for i, c in enumerate(vocab_answer)}

# Preguntas: Train
vocab_question = set()
for sentence in train_questions:
  for word in sentence:
    vocab_question.add(word)
vocab_question = ["#end"] + list(vocab_question)
print('Posibles palabras para preguntas (train): ', len(vocab_question))
vocabQTrain_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQTrain = {c: i for i, c in enumerate(vocab_question)}

print('Diferencia en la cantidad de palabras que componen las preguntas y respuestas (train sets): ', 
      abs(len(vocab_answer) - len(vocab_question)))

# Preguntas: Test
vocab_question = set()
for sentence in test_questions:
  for word in sentence:
    vocab_question.add(word)
vocab_question = ["#end"] + list(vocab_question)
print('Posibles palabras para preguntas (test): ', len(vocab_question))
vocabQTest_indices = {c: i for i, c in enumerate(vocab_question)}
indices_vocabQTest = {c: i for i, c in enumerate(vocab_question)}



Posibles palabras para respuestas:  47423
Posibles palabras para preguntas (train):  39482
Diferencia en la cantidad de palabras que componen las preguntas y respuestas (train sets):  7941
Posibles palabras para preguntas (test):  10322


El vocabulario de palabras que componen las respuestas tiene _7941_ elementos más que el que compone las preguntas, esto puede hacer que hayan palabras encontradas en preguntas asociadas a varias palabras de respuesta, haciendo más dificil el discernir la respuesta correcta. Por otro lado, se debe notar la pequeña cantidad de palabras que componen el vocabulario de test.

## d) Codificación de tokens y padding

Aplicaremos una codificación tipo *one-hot vector* sobre los tokens, calcularemos el largo máximo que puede tener una respuesta y una pregunta y reformularemos las secuencias de entrada del modelo agregandoles un padding al final, esto hará que el tamaño de input sea constante. Para las preguntas se rellenará con '0' (recordar que las palabras estan indexadas y tokenizadas), mientras que para las respuestas se rellenará con el carácter definido *'#end'* que indica cuando la pregunta ha sido respondida.



In [0]:
# input and output to onehotvector
X_answers = [[vocabA_indices[palabra] for palabra in sentence] for sentence in train_answers]
X_test_Q = [[vocabQTest_indices[palabra] for palabra in sentence] for sentence in test_questions]
X_train_Q = [[vocabQTrain_indices[palabra] for palabra in sentence] for sentence in train_questions]

# padding
max_input_lenght = np.max(list(map(len, train_questions)))
max_output_lenght = np.max(list(map(len, train_answers)))

X_train_Q = sequence.pad_sequences(X_train_Q, maxlen = max_input_lenght,
                                        padding = 'post', value = 0)
X_test_Q = sequence.pad_sequences(X_test_Q, maxlen = max_input_lenght,
                                        padding = 'post', value = 0)
X_answers = sequence.pad_sequences(X_answers, maxlen = max_output_lenght,
                                        padding = 'post', value = vocabA_indices['#end'])

## e) Modelo *Encoder-Decoder* con módulos de atención

In [0]:
# Encoder-Decoder modelo
lenght_output = max_output_lenght
hidden_dim = 128

